This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import numpy as np
import requests
import json 

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2017-01-01&end_date=2017-01-01&api_key=' + API_KEY
res = requests.get(url).json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(res)

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2020-12-01T14:48:09.907Z', 'newest_available_date': '2020-12-01', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2017-01-01', 'end_date': '2017-01-01', 'data': [], 'collapse': None, 'order': None, 'database_id': 6129}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# 1- Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?"+ "&start_date=2017-01-01&end_date=2017-12-31&api_key=" + API_KEY
res = requests.get(url)

In [6]:
# 2- Convert the returned JSON object into a Python dictionary.
res = res.json()
print(type(res))

<class 'dict'>


In [7]:
data_afx_x = res['dataset']['data']

In [8]:
# 3- Calculate what the highest and lowest opening prices were for the stock in this period.
price_open = [x[1] for x in data_afx_x if x[1] != None]
print("Highest opening price: " + str(max(price_open)))
print("Lowest opening price: " + str(min(price_open)))

Highest opening price: 53.11
Lowest opening price: 34.0


In [9]:
# 4- What was the largest change in any one day (based on High and Low price)?
price_high = [x[2] for x in data_afx_x if x[2] != None]
price_low = [x[3] for x in data_afx_x if x[3] != None]
price_change = [abs(h - l) for (h, l) in zip(price_high, price_low)]
print ("Largest change in any one day: " + str(max(price_change)))

Largest change in any one day: 2.8100000000000023


In [10]:
# 5- What was the largest change between any two days (based on Closing Price)?
price_close = [x[4] for x in data_afx_x if x[4] != None]
price_close_prev = [x[4] for x in data_afx_x if x[4] != None][1:]
price_close_diff = [abs(c - p) for (c, p) in zip(price_close, price_close_prev)]
print ("Largest change between two days: " + str(max(price_close_diff)))

Largest change between two days: 2.559999999999995


In [11]:
# 6- What was the average daily trading volume during this year?
trading_vol = [x[6] for x in data_afx_x]
trading_vol_avg = sum(trading_vol) / len(trading_vol)
print ("Avarage daily trading volume: " + str(round(trading_vol_avg,2)))

Avarage daily trading volume: 89124.34


In [12]:
# 7- (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
def calc_median(values):
    n = len(values)
    trading_vol_sorted = sorted(values, reverse= False)
    if n % 2 == 0:
        median_l = trading_vol_sorted[n//2]
        median_r = trading_vol_sorted[n//2 - 1]
        return (median_l + median_r)/2
    else:
        return trading_vol_sorted[n//2]
    
print('Median trading volume: ' + str(calc_median(trading_vol)))

Median trading volume: 76286.0
